In [10]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import backtrader.indicators as btind ## For indicators
import backtrader.feeds as btfeeds
import datetime  # For datetime objects


# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class BollingerBands(bt.Strategy):
    params = (('period', 30),('deviation', 2.0),)
    
    def log(self, text, dt = None):
        dt = dt or self.datas[0].datetime.date(0)
        print("%s %s" %(dt.isoformat(), text))
        
    def __init__(self):
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Add a BBand indicator
        self.bband = btind.BollingerBands(self.datas[0], period=self.p.period, devfactor=self.p.deviation)
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Reset
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %(trade.pnl, trade.pnlcomm))
    
    def next(self):
        self.log('Close, %.2f' % self.data.close[0])
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        # Check if we are in market
        if not self.position:
            # if the close value is lower than bollinger low it is a buying opportunity
            if self.data.close < self.bband.lines.bot:
                #Let's go long
                self.log('BUY CREATE, %.2f' % self.data.close[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
            # if the close value is greater than bollinger high it is a selling opportunity
            elif self.data.close > self.bband.lines.top:
                # SELL, SELL, SELL!!! (Since we are not in market Let's Short)
                self.log('SELL CREATE, %.2f' % self.data.close[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
        else:
            # exiting the long position Reaching the midline can be a good target
            if self.position.size>0 and self.data.close >= self.bband.lines.mid:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.data.close[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.close()
            
            elif self.position.size<0 and self.data.close <= self.bband.lines.mid:
                #Buy
                self.log('BUY CREATE, %.2f' % self.data.close[0])
                
                self.order = self.close()
                

if __name__ == "__main__":
    cerebro = bt.Cerebro()
    data = btfeeds.YahooFinanceCSVData(
        dataname = 'AMZN.csv',
        fromdate=datetime.datetime(2016, 9, 19),
        # Do not pass values after this date
        todate=datetime.datetime(2020, 9, 19),
        reverse=False
    )
    
    #Add data
    cerebro.adddata(data)
    #Add strategy
    cerebro.addstrategy(BollingerBands)
    #fix the size
    cerebro.addsizer(bt.sizers.SizerFix, stake=10)
    #initial investment
    cerebro.broker.setcash(100000.0)  
    #some money for the broker
    cerebro.broker.setcommission(commission=0.00)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2016-10-28 Close, 776.32
2016-10-28 BUY CREATE, 776.32
2016-10-31 BUY EXECUTED, Price: 781.03, Cost: 7810.30, Comm 0.00
2016-10-31 Close, 789.82
2016-11-01 Close, 785.41
2016-11-02 Close, 765.56
2016-11-03 Close, 767.03
2016-11-04 Close, 755.05
2016-11-07 Close, 784.93
2016-11-08 Close, 787.75
2016-11-09 Close, 771.88
2016-11-10 Close, 742.38
2016-11-11 Close, 739.01
2016-11-14 Close, 719.07
2016-11-15 Close, 743.24
2016-11-16 Close, 746.49
2016-11-17 Close, 756.40
2016-11-18 Close, 760.16
2016-11-21 Close, 780.00
2016-11-22 Close, 785.33
2016-11-23 Close, 780.12
2016-11-25 Close, 780.37
2016-11-28 Close, 766.77
2016-11-29 Close, 762.52
2016-11-30 Close, 750.57
2016-12-01 Close, 743.65
2016-12-02 Close, 740.34
2016-12-05 Close, 759.36
2016-12-06 Close, 764.72
2016-12-07 Close, 770.42
2016-12-07 SELL CREATE, 770.42
2016-12-08 SELL EXECUTED, Price: 771.87, Cost: 7810.30, Comm 0.00
2016-12-08 OPERATION PROFIT, GROSS -91.60, NET -91.60
2016-12-08 Close, 